# FX 图模式后训练静态量化

[教程](https://pytorch.org/tutorials/prototype/fx_graph_mode_ptq_static.html) 介绍了基于 {mod}`torch.fx` 在图模式下进行静态量化的步骤。

FX 图模式量化的优点：可以完全自动地对模型进行量化。

````{admonition} 大致流程
```python
import torch
from torch.ao.quantization import get_default_qconfig
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
from torch.ao.quantization import QConfigMapping
float_model.eval()
# 旧的 'fbgemm' 仍然可用，但 'x86' 是推荐的默认值。
qconfig = get_default_qconfig("x86")
qconfig_mapping = QConfigMapping().set_global(qconfig)
def calibrate(model, data_loader):
    model.eval()
    with torch.no_grad():
        for image, target in data_loader:
            model(image)
example_inputs = (next(iter(data_loader))[0]) # 获取样本输入
prepared_model = prepare_fx(float_model, qconfig_mapping, example_inputs)  # 融合模块并插入观测器
calibrate(prepared_model, data_loader_test)  # 在样本数据上进行校准
quantized_model = convert_fx(prepared_model)  # 将校准模型转换为量化模型
```
````

## FX 图模式量化的动机

目前，PyTorch 只有 eager 模式量化作为替代：[静态量化](https://pytorch.org/tutorials/advanced/static_quantization_tutorial.html)。

eager 模式量化过程中涉及到多个手动步骤，包括：

- 显式量化和反量化激活——当模型中时同时存在浮点运算和量化运算，这很耗时。
- 显式地融合模块——这需要手动识别卷积序列、batch norms 和 relus 以及其他融合模式。
- Pytorch 张量运算（如 `add`，`concat` 等）需要特殊处理。
- 函数没有 first class 的支持（`functional.conv2d` 和 `functional.linear` 不会被量化）

这些需要修改的大部分都来自 eager 模式量化的潜在局限性。Eager 模式工作在模块级别，因为它不能检查实际运行的代码（{func}`forward` 函数），量化是通过模块交换来实现的，不知道 {func}`forward` 函数中的模块是如何使用的，所以它需要用户手动插入 QuantStub 和 DeQuantStub 来标记想要量化或反量化的点。在图模式中，可以检查在 {func}`forward` 函数中执行的实际代码（例如，{func}`aten` 函数调用），量化是通过模块和图操作实现的。由于图模式对正在运行的代码具有完全可见性，这样能够自动地找出哪些模块需要融合，在哪里插入观测器调用，量化/反量化函数等，能够自动化整个量化过程。

FX 图模式量化的优点是：

- 简化量化流程，最小的手工步骤
- 解锁了进行更高级别优化的可能性，如自动精度选择

## 定义辅助函数并准备数据集

定义辅助函数：

In [1]:
import os
from copy import deepcopy
import torch
from torch.ao.quantization import get_default_qconfig, QConfigMapping
from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx, fuse_fx
from torch import nn

# 为可重复的结果指定随机种子
_ = torch.manual_seed(191009)

In [2]:
from dataclasses import dataclass


@dataclass
class AverageMeter:
    """计算并存储平均值和当前值"""
    name: str
    fmt: str = ":f"
    def __post_init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

def evaluate(model, data_loader):
    model.eval()
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            output = model(image)
            # criterion(output, target) # 计算损失
            cnt += 1
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            top1.update(acc1[0], image.size(0))
            top5.update(acc5[0], image.size(0))
    return top1, top5

def size_of_model(model):
    """返回模型大小"""
    torch.save(model.state_dict(), "temp.p")
    size = os.path.getsize('temp.p')
    os.remove("temp.p")
    return size

准备数据：

In [3]:
from dataclasses import dataclass
from pathlib import Path
from torchvision.datasets.folder import ImageFolder
from torchvision import transforms


@dataclass
class Dataset:
    root: str

    def __post_init__(self):
        self.root = Path(self.root)
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])

    @property
    def trainset(self):
        return ImageFolder(self.root/"train", transform=transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            self.normalize]))

    @property
    def testset(self):
        return ImageFolder(self.root/"val", transform=transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            self.normalize]))

    def trainloader(self, batch_size):
        train_sampler = torch.utils.data.RandomSampler(self.trainset)
        return torch.utils.data.DataLoader(self.trainset, batch_size=batch_size, sampler=train_sampler)

    def testloader(self, batch_size):
        test_sampler = torch.utils.data.SequentialSampler(self.testset)
        return torch.utils.data.DataLoader(self.testset, batch_size=batch_size, sampler=test_sampler)

准备浮点模型：

In [4]:
from torchvision.models.resnet import resnet18, ResNet18_Weights


saved_model_dir = 'data/' # 模型存储路径
train_batch_size = 30 # 训练样本批量大小
eval_batch_size = 50 # 测试样本批量大小
dataset = Dataset('/media/pc/data/lxw/home/data/datasets/ILSVRC')
data_loader = dataset.trainloader(train_batch_size)
data_loader_test = dataset.testloader(eval_batch_size)
example_inputs = (next(iter(data_loader))[0])
criterion = nn.CrossEntropyLoss()
float_model = resnet18(weights=ResNet18_Weights.DEFAULT)
float_model = float_model.to("cpu").eval()

In [5]:
# 深度复制模型，因为需要保持原始模型
model_to_quantize = deepcopy(float_model)

## 设置模型为评估模式

对于训练后量化，需要将模型设置为评估模式。

In [6]:
model_to_quantize.eval();

## 使用 QConfigMapping 指定如何量化模型

```python
qconfig_mapping = QConfigMapping.set_global(default_qconfig)
```

使用与 eager 模式量化相同的 `qconfig`, `qconfig` 只是用于激活和权重的观测器的命名元组。`QConfigMapping` 包含 `ops` 到 `qconfigs` 的映射信息：

```python
model_to_quantize.eval()
qconfig_mapping = (QConfigMapping()
    .set_global(qconfig_opt)  # qconfig_opt 是可选的 qconfig，可以是有效的 qconfig，也可以是 None
    .set_object_type(torch.nn.Conv2d, qconfig_opt) # 可以是可调用的
    .set_object_type("torch.nn.functional.add", qconfig_opt) # 或者类名的字符串
    .set_module_name_regex("foo.*bar.*conv[0-9]+", qconfig_opt) # 按顺序匹配，第一个匹配优先
    .set_module_name("foo.bar", qconfig_opt)
    .set_module_name_object_type_order()
)
```
```{note}
- 优先级（按递增顺序）：global、object_type、module_name_regex、module_name
- `qconfig == None` 表示任何事情都应该跳过融合和量化
- 匹配规则（除非找到更高优先级的匹配）
```

与 `qconfig` 相关的实用函数可以在 [`qconfig`](https://github.com/pytorch/pytorch/blob/master/torch/ao/quantization/qconfig.py) 文件中找到，而与 `QConfigMapping` 相关的实用函数可以在 [`qconfig_mapping`](https://github.com/pytorch/pytorch/blob/master/torch/ao/quantization/fx/qconfig_mapping.py) 中找到。

In [7]:
# 旧的 'fbgemm' 仍然可用，但 'x86' 是推荐的默认值。
qconfig = get_default_qconfig("x86")
qconfig_mapping = QConfigMapping().set_global(qconfig)

## 为训练后静态量化准备模型

```python
prepared_model = prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)
```

`prepare_fx` 将 `BatchNorm` 模块折叠到其前面的 Conv2d 模块中，并在模型的适当位置插入观测器。

In [8]:
prepared_model = prepare_fx(model_to_quantize, qconfig_mapping, example_inputs)
print(prepared_model.graph)

graph():
    %x : torch.Tensor [#users=1] = placeholder[target=x]
    %activation_post_process_0 : [#users=1] = call_module[target=activation_post_process_0](args = (%x,), kwargs = {})
    %conv1 : [#users=1] = call_module[target=conv1](args = (%activation_post_process_0,), kwargs = {})
    %activation_post_process_1 : [#users=1] = call_module[target=activation_post_process_1](args = (%conv1,), kwargs = {})
    %maxpool : [#users=1] = call_module[target=maxpool](args = (%activation_post_process_1,), kwargs = {})
    %activation_post_process_2 : [#users=2] = call_module[target=activation_post_process_2](args = (%maxpool,), kwargs = {})
    %layer1_0_conv1 : [#users=1] = call_module[target=layer1.0.conv1](args = (%activation_post_process_2,), kwargs = {})
    %activation_post_process_3 : [#users=1] = call_module[target=activation_post_process_3](args = (%layer1_0_conv1,), kwargs = {})
    %layer1_0_conv2 : [#users=1] = call_module[target=layer1.0.conv2](args = (%activation_post_process_3

/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


## 校准

在将观测器插入模型后运行校准函数。校准的目的是运行一些代表工作负载的样本示例（例如训练数据集的样本），以便模型中的观测器能够观察到张量的统计信息，然后使用这些信息来计算量化参数。

In [9]:
def calibrate(model, data_loader, num=200):
    model.eval()
    with torch.no_grad():
        for k, (image, _) in enumerate(data_loader):
            if k > num:
                break
            model(image)
calibrate(prepared_model, data_loader)  # 在样本数据上运行校准

## 将模型转换为量化模型

`convert_fx` 采用校准模型并生成量化模型。

In [10]:
quantized_model = convert_fx(prepared_model)
print(quantized_model.graph)

graph():
    %x : torch.Tensor [#users=1] = placeholder[target=x]
    %conv1_input_scale_0 : [#users=1] = get_attr[target=conv1_input_scale_0]
    %conv1_input_zero_point_0 : [#users=1] = get_attr[target=conv1_input_zero_point_0]
    %quantize_per_tensor : [#users=1] = call_function[target=torch.quantize_per_tensor](args = (%x, %conv1_input_scale_0, %conv1_input_zero_point_0, torch.quint8), kwargs = {})
    %conv1 : [#users=1] = call_module[target=conv1](args = (%quantize_per_tensor,), kwargs = {})
    %maxpool : [#users=2] = call_module[target=maxpool](args = (%conv1,), kwargs = {})
    %layer1_0_conv1 : [#users=1] = call_module[target=layer1.0.conv1](args = (%maxpool,), kwargs = {})
    %layer1_0_conv2 : [#users=1] = call_module[target=layer1.0.conv2](args = (%layer1_0_conv1,), kwargs = {})
    %layer1_0_relu_scale_0 : [#users=1] = get_attr[target=layer1_0_relu_scale_0]
    %layer1_0_relu_zero_point_0 : [#users=1] = get_attr[target=layer1_0_relu_zero_point_0]
    %add_relu : [#users=

## 评估

打印量化模型的大小和精度。

In [11]:
print(f"量化前模型大小: {size_of_model(float_model)/(1<<20)} MB")
print(f"量化后模型大小: {size_of_model(quantized_model)/(1<<20)} MB")

量化前模型大小: 44.65853214263916 MB
量化后模型大小: 11.283549308776855 MB


In [12]:
top1, top5 = evaluate(quantized_model, data_loader_test)
print(f"[序列化前] 测试数据集的准确性评估: {top1.avg: 2.2g}, {top5.avg: 2.2g}")
fx_graph_mode_model_file_path = saved_model_dir + "resnet18_fx_graph_mode_quantized.pth"
# this does not run due to some erros loading convrelu module:
# ModuleAttributeError: 'ConvReLU2d' object has no attribute '_modules'
# save the whole model directly
# torch.save(quantized_model, fx_graph_mode_model_file_path)
# loaded_quantized_model = torch.load(fx_graph_mode_model_file_path)

# save with state_dict
# torch.save(quantized_model.state_dict(), fx_graph_mode_model_file_path)
# import copy
# model_to_quantize = copy.deepcopy(float_model)
# prepared_model = prepare_fx(model_to_quantize, {"": qconfig})
# loaded_quantized_model = convert_fx(prepared_model)
# loaded_quantized_model.load_state_dict(torch.load(fx_graph_mode_model_file_path))

# 保存 script 模型
torch.jit.save(torch.jit.script(quantized_model), fx_graph_mode_model_file_path)
loaded_quantized_model = torch.jit.load(fx_graph_mode_model_file_path)
top1, top5 = evaluate(loaded_quantized_model, data_loader_test)
print(f"[序列化后] 测试数据集的准确性评估: {top1.avg: 2.5g}, {top5.avg: 2.5g}")

[序列化前] 测试数据集的准确性评估:  69,  89


/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/torch/jit/_check.py:172: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


[序列化后] 测试数据集的准确性评估:  69.464,  88.936


如果想获得更好的精度或性能，请尝试更改 `qconfig_mapping`。

## 调试量化模型

还可以打印量化的非量化卷积运算的权值来查看差异，首先显式调用 `fuse` 来融合卷积和模型中的批处理范数：注意 `fuse_fx` 只在 eval 模式下工作。

In [13]:
fused = fuse_fx(float_model)
conv1_weight_after_fuse = fused.conv1[0].weight[0]
conv1_weight_after_quant = quantized_model.conv1.weight().dequantize()[0]
print(torch.max(abs(conv1_weight_after_fuse - conv1_weight_after_quant)))

tensor(0.0007, grad_fn=<MaxBackward1>)


## 与基线浮点模型和 eager 模式量化的比较

In [14]:
scripted_float_model_file = "resnet18_scripted.pth"
print(f"baseline 模型大小: {size_of_model(float_model)/(1<<20)} MB")
top1, top5 = evaluate(float_model, data_loader_test)
print(f"baseline 浮点模型评估精度: {top1.avg: 2.5g}, {top5.avg: 2.5g}")
torch.jit.save(torch.jit.script(float_model), saved_model_dir + scripted_float_model_file)

baseline 模型大小: 44.65853214263916 MB
baseline 浮点模型评估精度:  69.758,  89.078


在本节中，比较了用 FX 图模式量化的模型和用 eager 模式量化的模型。FX 图模式和 eager 模式产生非常相似的量化模型，因此期望精度和加速也是相似的。

In [15]:
print(f"FX 图模式量化模型大小: {size_of_model(quantized_model)/(1<<20)} MB")
top1, top5 = evaluate(quantized_model, data_loader_test)
print(f"FX 图模式量化评估精度: {top1.avg: 2.5g}, {top5.avg: 2.5g}")

from torchvision.models.quantization.resnet import resnet18
eager_quantized_model = resnet18(pretrained=True, quantize=True).eval()
print("Size of eager mode quantized model")
eager_quantized_model = torch.jit.script(eager_quantized_model)
print(f"eager 模式量化模型大小: {size_of_model(eager_quantized_model)/(1<<20)} MB")
top1, top5 = evaluate(eager_quantized_model, data_loader_test)
print(f"eager 模式量化评估精度: {top1.avg: 2.5g}, {top5.avg: 2.5g}")
eager_mode_model_file = "resnet18_eager_mode_quantized.pth"
torch.jit.save(eager_quantized_model, saved_model_dir + eager_mode_model_file)

FX 图模式量化模型大小: 11.283549308776855 MB
FX 图模式量化评估精度:  69.464,  88.936


/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_QuantizedWeights.IMAGENET1K_FBGEMM_V1`. You can also use `weights=ResNet18_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/media/pc/data/tmp/cache/conda/envs/tvmz/lib/python3.10/site-packages/torch/_utils.py:320: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To acces

Size of eager mode quantized model
eager 模式量化模型大小: 0.0035047531127929688 MB


可以看到，FX 图模式和 eager 模式量化模型的模型大小和精度非常相似。

正如我们所看到的，对于 resnet18, FX 图模式和 eager 模式量化模型都比浮点模型得到了相似的加速，比浮点模型快 2-4 倍左右。但是浮点模型的实际加速可能会根据模型、设备、构建、输入批大小、线程等而有所不同。